In [4]:
!pip install -r requirements.txt

### Import the video prompter from cuts

In [1]:
from video_prompter import get_video, video_prompter
from videodb import play_stream

### Choose the input video

In [51]:
from videodb import play_stream
from video_prompter import get_video, video_prompter, get_connection
from dotenv import load_dotenv
from videodb.timeline import Timeline, VideoAsset, AudioAsset
from videodb import MediaType
import os

# TODO: setup .env file
load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
conn = get_connection()

### ----- Upload a fresh video --------- #####
def fresh_video(url):
    video = conn.upload(url)
    #index spoken content in the video
    video.index_spoken_words()
    return video

#### ------ Pick exisiting video from your VideoDB --------####
def videodb_prompter(video_id, prompt):
    video = get_video(video_id)
    #get all the segment of videos that are
    return video_prompter(video, prompt)

### Decide your prompt

In [43]:
prompt = "find the moments useful for social media trailer for this video"

# Existing video case
# TODO: replace with your video id
# video_id = "m-replace-with-your-video-id-24-7"

# Fresh video case
# url = ""
# video = fresh_video(url)
# video_id = video.id

timestamps = videodb_prompter(video_id, prompt)

{'id': 'chatcmpl-8twtL6K6sZlcQDypoDKQiYvqtcMbz', 'object': 'chat.completion', 'created': 1708345111, 'model': 'gpt-3.5-turbo-16k-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "timestamps": [\n    {\n      "s": 997.5,\n      "e": 1003.9\n    },\n    {\n      "s": 1003.4,\n      "e": 1018.8\n    },\n    {\n      "s": 1018.5,\n      "e": 1029.6\n    },\n    {\n      "s": 1028.0,\n      "e": 1038.6\n    },\n    {\n      "s": 1038.2,\n      "e": 1046.2\n    },\n    {\n      "s": 1045.9,\n      "e": 1058.4\n    },\n    {\n      "s": 1055.6,\n      "e": 1063.2\n    },\n    {\n      "s": 1063.1,\n      "e": 1086.4\n    },\n    {\n      "s": 1086.4,\n      "e": 1087.0\n    },\n    {\n      "s": 1086.4,\n      "e": 1095.7\n    },\n    {\n      "s": 1095.4,\n      "e": 1107.7\n    },\n    {\n      "s": 1107.3,\n      "e": 1112.2\n    },\n    {\n      "s": 1112.1,\n      "e": 1124.7\n    },\n    {\n      "s": 1124.4,\n      "e": 1133.8\n    },\n    {\n      "s":

### Rank your results

You can control the time, quality etc. of each chunk selected. We are using a very simple critaria here.

In [9]:
def process_timestamps(timestamps):
    """
    Sort, reject, and merge timestamp chunks based on specified criteria.
    - Reject chunks with identical start and end times or where start is greater than end.
    - Discard chunks shorter than 5 seconds.
    - Merge overlapping chunks.
    :param timestamps: List of timestamp dictionaries with 's' and 'e' as keys.
    :return: List of tuples representing the merged and filtered timeline.
    """
    # Filter out invalid timestamps.
    filtered_timestamps = [ts for ts in timestamps if ts['s'] < ts['e'] and (ts['e'] - ts['s']) >= 5]

    # Sort the timestamps by start time.
    sorted_timestamps = sorted(filtered_timestamps, key=lambda x: x['s'])

    if len(sorted_timestamps) == 0:
        return []

    # Initialize the stack with the first timestamp.
    stack = [sorted_timestamps[0]]

    # Iterate over the rest of the timestamps.
    for i in range(1, len(sorted_timestamps)):
        top = stack[-1]

        # Check for overlap with the top of the stack; merge if necessary.
        if sorted_timestamps[i]['s'] <= top['e']:
            merged_end = max(top['e'], sorted_timestamps[i]['e'])
            stack[-1] = {'s': top['s'], 'e': merged_end}  # Update the top element with the merged time.
        else:
            stack.append(sorted_timestamps[i])  # No overlap; add to the stack.

    # Convert to list of tuples for the return value.
    merged_timeline = [(item['s'], item['e']) for item in stack]
    return merged_timeline

In [44]:
process_timestamps(timestamps)

[(997.5, 1107.7),
 (1112.1, 1215.9),
 (1220.2, 1299.8),
 (1303.6, 1449.3),
 (1453.6, 1542.0)]

### Choose the order and generate stream

Here we generated the new timeline, in same order as the sorted timestamps, we didn't drop any of the segment and didn't verify it but you should use your own logic to decide on order of adding them inline. 

In [45]:
timeline = Timeline(conn)
dur_so_far = 0
for segment in process_timestamps(timestamps):
    # here you can rank each asset again for your prompt and decide order,
    video_asset = VideoAsset(asset_id=video_id, start=segment[0], end=segment[1])
    timeline.add_inline(video_asset)
    dur_so_far += (segment[1] - segment[0])

In [46]:
stream = timeline.generate_stream()
play_stream(stream)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/93b9cb43-6747-4a1b-95aa-46cbd715ae86.m3u8'

### Add some sound effects to it 🎶

Not just this we can jazz it up with audio overlays and create another stream. 

In [48]:
# Add music overlay, this can be laughter soundtrack
audio = conn.upload(file_path="laughter_sound.mp3", media_type=MediaType.audio)

In [47]:
# 5 sec background music
background = AudioAsset(asset_id=audio.id, start=0, disable_other_tracks=True)

In [49]:
timeline = Timeline(conn)
dur_so_far = 0
for segment in process_timestamps(timestamps_):
    # here you can rank each asset again for your prompt and decide order,
    video_asset = VideoAsset(asset_id=video_id, start=segment[0], end=segment[1])
    timeline.add_inline(video_asset)
    dur_so_far += (segment[1] - segment[0])

In [50]:
#add music overlay in the last 2 sec of the supercut.
timeline.add_overlay(20,background)
stream = timeline.generate_stream()
play_stream(stream)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/9e10504f-b2b0-4fec-8efb-22075d68338c.m3u8'